In [11]:
import random
import math    # cos() for Rastrigin
import copy    # array-copying convenience
import sys     # max float
 
 
#-------fitness functions---------
 
# rastrigin function
def fitness_rastrigin(position):
  fitness_value = 0.0
  for i in range(len(position)):
    xi = position[i]
    fitness_value += (xi * xi) - (10 * math.cos(2 * math.pi * xi)) + 10
  return fitness_value
 
#sphere function
def fitness_sphere(position):
    fitness_value = 0.0
    for i in range(len(position)):
        xi = position[i]
        fitness_value += (xi*xi);
    return fitness_value;
#-------------------------





#Student class
class Student:
  def __init__(self, fitness, dim, minx, maxx, seed):
    self.rnd = random.Random(seed)
 
    # a list of size dim
    # with 0.0 as value of all the elements
    self.position = [0.0 for i in range(dim)]
 
    # loop dim times and randomly select value of decision var
    # value should be in between minx and maxx
    for i in range(dim):
      self.position[i] = ((maxx - minx) *
        self.rnd.random() + minx)
 
    # compute the fitness of student
    self.fitness = fitness(self.position)
 
 
# Teaching learning based optimization
def tlbo(fitness, max_iter, n, dim, minx, maxx):
  rnd = random.Random(0)
 
  # create n random students
  classroom = [Student(fitness, dim, minx, maxx, i) for i in range(n)]
 
  # compute the value of best_position and best_fitness in the classroom
  Xbest = [0.0 for i in range(dim)]
  Fbest = sys.float_info.max       
 
  for i in range(n): # check each Student
    if classroom[i].fitness < Fbest:
      Fbest = classroom[i].fitness
      Xbest = copy.copy(classroom[i].position)
 
  # main loop of tlbo
  Iter = 0
  while Iter < max_iter:
     
    # after every 10 iterations
    # print iteration number and best fitness value so far
    if Iter % 10 == 0 and Iter > 1:
      print("Iter = " + str(Iter) + " best fitness = %.3f" % Fbest)
 
    # for each student of classroom
    for i in range(n):
 
      ### Teaching phase of ith student
 
      # compute the mean of all the students in the class
      Xmean = [0.0 for i in range(dim)]
      for k in range(n):
          for j in range(dim):
              Xmean[j]+= classroom[k].position[j]
       
      for j in range(dim):
          Xmean[j]/= n;
       
      # initialize new solution
      Xnew = [0.0 for i in range(dim)]
 
      # teaching factor (TF)
      # either 1 or 2 ( randomly chosen)
      TF = random.randint(1, 3)
 
      # best student of the class is teacher
      Xteacher = Xbest
 
      # compute new solution
      for j in range(dim):
          Xnew[j] = classroom[i].position[j] + rnd.random()*(Xteacher[j] - TF*Xmean[j])
       
      # if Xnew < minx OR Xnew > maxx
      # then clip it
      for j in range(dim):
          Xnew[j] = max(Xnew[j], minx)
          Xnew[j] = min(Xnew[j], maxx)
       
      # compute fitness of new solution
      fnew = fitness(Xnew)
 
      # if new solution is better than old
      # replace old with new solution
      if(fnew < classroom[i].fitness):
          classroom[i].position = Xnew
          classroom[i].fitness = fnew
        
      # update best student
      if(fnew < Fbest):
          Fbest = fnew
          Xbest = Xnew
 
 
 
      ### learning phase of ith student
 
      # randomly choose a solution from classroom
      # chosen solution should not be ith student
      p = random.randint(0, n-1)
      while(p==i):
          p = random.randint(0, n-1)
       
      # partner solution
      Xpartner = classroom[p]
 
      Xnew = [0.0 for i in range(dim)]
      if(classroom[i].fitness < Xpartner.fitness):
          for j in range(dim):
              Xnew[j] = classroom[i].position[j] + rnd.random()*(classroom[i].position[j] - Xpartner.position[j])
      else:
          for j in range(dim):
              Xnew[j] = classroom[i].position[j] - rnd.random()*(classroom[i].position[j] - Xpartner.position[j])
 
      # if Xnew < minx OR Xnew > maxx
      # then clip it
      for j in range(dim):
          Xnew[j] = max(Xnew[j], minx)
          Xnew[j] = min(Xnew[j], maxx)
       
      # compute fitness of new solution
      fnew = fitness(Xnew)
 
      # if new solution is better than old
      # replace old with new solution
      if(fnew < classroom[i].fitness):
          classroom[i].position = Xnew
          classroom[i].fitness = fnew
        
      # update best student
      if(fnew < Fbest):
          Fbest = fnew
          Xbest = Xnew
 
    Iter += 1
  # end-while
 
  # return best student from classroom
  return Xbest
# end pso
 
 
#----------------------------
# Driver code for rastrigin function
 
print("\nBegin  teaching learning based optimization on rastrigin function\n")
dim = 3
fitness = fitness_rastrigin
 
 
print("Goal is to minimize Rastrigin's function in " + str(dim) + " variables")
print("Function has known min = 0.0 at (", end="")
for i in range(dim-1):
  print("0, ", end="")
print("0)")
 
num_particles = 50
max_iter = 100
 
print("Setting num_particles = " + str(num_particles))
print("Setting max_iter    = " + str(max_iter))
print("\nStarting TLBO algorithm\n")
 
 
 
best_position = tlbo(fitness, max_iter, num_particles, dim, -10.0, 10.0)
 
print("\nTLBO completed\n")
print("\nBest Student found:")
print(["%.6f"%best_position[k] for k in range(dim)])
fitness_value = fitness(best_position)
print("fitness of best Student = %.6f" % fitness_value)
 
print("\nEnd TLBO for rastrigin function\n")
 
 
print()
print()
 
 
# Driver code for Sphere function
print("\nBegin teaching learning based optimization on sphere function\n")
dim = 3
fitness = fitness_sphere
 
 
print("Goal is to minimize sphere function in " + str(dim) + " variables")
print("Function has known min = 0.0 at (", end="")
for i in range(dim-1):
  print("0, ", end="")
print("0)")
 
num_particles = 50
max_iter = 100
 
print("Setting num_particles = " + str(num_particles))
print("Setting max_iter    = " + str(max_iter))
print("\nStarting TLBO algorithm\n")
 
 
 
best_position = tlbo(fitness, max_iter, num_particles, dim, -10.0, 10.0)
 
print("\nTLBO completed\n")
print("\nBest Student found:")
print(["%.6f"%best_position[k] for k in range(dim)])
fitness_value = fitness(best_position)
print("fitness of best Student = %.6f" % fitness_value)
 
print("\nEnd TLBO for sphere function\n")


Begin  teaching learning based optimization on rastrigin function

Goal is to minimize Rastrigin's function in 3 variables
Function has known min = 0.0 at (0, 0, 0)
Setting num_particles = 50
Setting max_iter    = 100

Starting TLBO algorithm

Iter = 10 best fitness = 2.706
Iter = 20 best fitness = 1.481
Iter = 30 best fitness = 1.307
Iter = 40 best fitness = 1.124
Iter = 50 best fitness = 1.029
Iter = 60 best fitness = 0.386
Iter = 70 best fitness = 0.386
Iter = 80 best fitness = 0.048
Iter = 90 best fitness = 0.023

TLBO completed


Best Student found:
['0.002457', '-0.003317', '-0.000565']
fitness of best Student = 0.003444

End TLBO for rastrigin function




Begin teaching learning based optimization on sphere function

Goal is to minimize sphere function in 3 variables
Function has known min = 0.0 at (0, 0, 0)
Setting num_particles = 50
Setting max_iter    = 100

Starting TLBO algorithm

Iter = 10 best fitness = 0.004
Iter = 20 best fitness = 0.000
Iter = 30 best fitness = 0.000

In [268]:
import random
import math    # cos() for Rastrigin
import copy    # array-copying convenience
import sys     # max float
from math import sqrt
#rastrigin funcction
def fitness_rastrigin(position):
    fitness_value = 0.0
    for i in range(len(position)):
        xi = position[i]
        fitness_value += (xi * xi) - (10 * math.cos(2 * math.pi * xi)) + 10
    return fitness_value
 

#sphere function
def fitness_sphere(position):
    fitness_value = 0.0
    for i in range(len(position)):
        xi = position[i]
        fitness_value += (xi*xi)
    return fitness_value

#benchmark 1
def fitness_bench1(position):
    fitness_value = 0.0
    for i in range(len(position)):
        xi = position[i]
        fitness_value += 5 * xi + 5 * (xi * xi)
    return fitness_value

#benchmark 2
def fitness_bench2(position):
    fitness_value = 0.0
    for i in range(len(position)):
        xi = position[i]
        fitness_value += (math.sqrt(i)**i) * xi
    return fitness_value

#benchmark 3
def fitness_bench3(position):
    fitness_value = 0.0
    for i in range(len(position)):
        xi = position[i]
        fitness_value += (xi - 10)**2 + (xi - 12)**2 + 3*(xi-11)**2 + 10*xi**6 + xi*4 - 4*xi**2 - 18 * xi
    return fitness_value

#benchmark 4
def fitness_bench4(position):
    fitness_value = 0.0
    for i in range(len(position)):
        xi = position[i]
        fitness_value += xi + (xi**2) + (xi**3)
    return fitness_value

#benchmark 5
def fitness_bench5(position):
    fitness_value = 0.0
    for i in range(len(position)):
        xi = position[i]
        fitness_value += xi + (xi**2) + (xi**3)
    return fitness_value

#benchmark 4
def fitness_bench4(position):
    fitness_value = 0.0
    for i in range(len(position)):
        xi = position[i]
        fitness_value += xi + (xi**2) + (xi**3)
    return fitness_value

In [145]:
class Student:
    def __init__(self, fitness, dim, minx, maxx, seed):
        self.rnd = random.Random(seed)
        self.position = [0.0 for i in range(dim)]
        for i in range(dim):
            self.position[i] = ((maxx - minx) * self.rnd.random() + minx)
        self.fitness = fitness(self.position)
    
    def tlbo(fitness, max_iter, n, dim, minx, maxx):
        rnd = random.Random(0)
        classroom = [Student(fitness, dim, minx, maxx, i) for i in range(n)]
        Xbest = [0.0 for i in range(dim)]
        Fbest = sys.float_info.max       
     
        for i in range(n):
            if classroom[i].fitness < Fbest:
                Fbest = classroom[i].fitness
                Xbest = copy.copy(classroom[i].position)
    
        Iter = 0
        while Iter < max_iter:
            if Iter % 10 == 0 and Iter > 1:
                print("Iter = " + str(Iter) + " best fitness = %.3f" % Fbest)
    
            for i in range(n):
                Xmean = [0.0 for i in range(dim)]
            
            for k in range(n):
                for j in range(dim):
                    Xmean[j]+= classroom[k].position[j]
           
            for j in range(dim):
                Xmean[j]/= n;
    
            Xnew = [0.0 for i in range(dim)]
    
            TF = random.randint(1, 3)
    
            Xteacher = Xbest
    
            for j in range(dim):
                Xnew[j] = classroom[i].position[j] + rnd.random()*(Xteacher[j] - TF*Xmean[j])
    
            for j in range(dim):
                Xnew[j] = max(Xnew[j], minx)
                Xnew[j] = min(Xnew[j], maxx)
    
            fnew = fitness(Xnew)
    
            if(fnew < classroom[i].fitness):
                classroom[i].position = Xnew
                classroom[i].fitness = fnew
    
            if(fnew < Fbest):
                Fbest = fnew
                Xbest = Xnew
    
            p = random.randint(0, n-1)
            while(p==i):
                p = random.randint(0, n-1)
    
            Xpartner = classroom[p]
     
            Xnew = [0.0 for i in range(dim)]
            if(classroom[i].fitness < Xpartner.fitness):
                for j in range(dim):
                    Xnew[j] = classroom[i].position[j] + rnd.random()*(classroom[i].position[j] - Xpartner.position[j])
            else:
                for j in range(dim):
                    Xnew[j] = classroom[i].position[j] - rnd.random()*(classroom[i].position[j] - Xpartner.position[j])
    
            for j in range(dim):
                Xnew[j] = max(Xnew[j], minx)
                Xnew[j] = min(Xnew[j], maxx)
    
            fnew = fitness(Xnew)
    
            if(fnew < classroom[i].fitness):
                classroom[i].position = Xnew
                classroom[i].fitness = fnew
    
            if(fnew < Fbest):
                Fbest = fnew
                Xbest = Xnew
     
        Iter += 1
    
        return Xbest

In [234]:
#rastrigin 
print("\nBegin  teaching learning based optimization on rastrigin function\n")
dim = 10
fitness = fitness_rastrigin
 
print("Goal is to minimize Rastrigin's function in " + str(dim) + " variables")
print("Function has known min = 0.0 at (", end="")
for i in range(dim-1):
    print("0, ", end="")
print("0)")
 
num_particles = 50
max_iter = 100
 
print("Setting num_particles = " + str(num_particles))
print("Setting max_iter    = " + str(max_iter))
print("\nStarting TLBO algorithm\n")
 
 
 
best_position = tlbo(fitness, max_iter, num_particles, dim, -10.0, 10.0)
 
print("\nTLBO completed\n")
print("\nBest Student found:")
print(["%.6f"%best_position[k] for k in range(dim)])
fitness_value = fitness(best_position)
print("fitness of best Student = %.6f" % fitness_value)
 
print("\nEnd TLBO for rastrigin function\n")


Begin  teaching learning based optimization on rastrigin function

Goal is to minimize Rastrigin's function in 10 variables
Function has known min = 0.0 at (0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
Setting num_particles = 50
Setting max_iter    = 100

Starting TLBO algorithm

Iter = 10 best fitness = 42.033
Iter = 20 best fitness = 39.260
Iter = 30 best fitness = 36.441
Iter = 40 best fitness = 29.112
Iter = 50 best fitness = 25.790
Iter = 60 best fitness = 18.720
Iter = 70 best fitness = 17.880
Iter = 80 best fitness = 14.515
Iter = 90 best fitness = 11.521

TLBO completed


Best Student found:
['0.915387', '-0.032847', '-0.093170', '-0.028907', '0.066341', '-0.978859', '0.063254', '0.992441', '-0.902080', '-0.068493']
fitness of best Student = 11.521083

End TLBO for rastrigin function



In [260]:
#Sphere function
print("\nBegin teaching learning based optimization on sphere function\n")
dim = 13
fitness = fitness_sphere
 
print("Goal is to minimize sphere function in " + str(dim) + " variables")
print("Function has known min = 0.0 at (", end="")
for i in range(dim-1):
    print("0, ", end="")
print("0)")
 
num_particles = 50
max_iter = 100
 
print("Setting num_particles = " + str(num_particles))
print("Setting max_iter    = " + str(max_iter))
print("\nStarting TLBO algorithm\n")
 
 
 
best_position = tlbo(fitness, max_iter, num_particles, dim, -10.0, 10.0)
 
print("\nTLBO completed\n")
print("\nBest Student found:")
print(["%.6f"%best_position[k] for k in range(dim)])
fitness_value = fitness(best_position)
print("fitness of best Student = %.6f" % fitness_value)
 
print("\nEnd TLBO for sphere function\n")


Begin teaching learning based optimization on sphere function

Goal is to minimize sphere function in 13 variables
Function has known min = 0.0 at (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
Setting num_particles = 50
Setting max_iter    = 100

Starting TLBO algorithm

Iter = 10 best fitness = 3.036
Iter = 20 best fitness = 0.008
Iter = 30 best fitness = 0.000
Iter = 40 best fitness = 0.000
Iter = 50 best fitness = 0.000
Iter = 60 best fitness = 0.000
Iter = 70 best fitness = 0.000
Iter = 80 best fitness = 0.000
Iter = 90 best fitness = 0.000

TLBO completed


Best Student found:
['0.000000', '0.000000', '0.000000', '0.000000', '-0.000000', '0.000000', '-0.000000', '-0.000000', '-0.000000', '-0.000000', '-0.000000', '0.000000', '-0.000000']
fitness of best Student = 0.000000

End TLBO for sphere function



In [203]:
#rastrigin 
print("\nBegin  teaching learning based optimization on rastrigin function\n")
dim = 13
fitness = fitness_bench1
 
print("Goal is to minimize Rastrigin's function in " + str(dim) + " variables")
print("Function has known min = 0.0 at (", end="")
for i in range(dim-1):
    print("0, ", end="")
print("0)")
 
num_particles = 50 # population size (no. of student)
max_iter = 100
 
print("Setting num_particles = " + str(num_particles))
print("Setting max_iter    = " + str(max_iter))
print("\nStarting TLBO algorithm\n")
 
 
 
best_position = tlbo(fitness, max_iter, num_particles, dim, -10.0, 10.0)
 
print("\nTLBO completed\n")
print("\nBest Student found:")
print(["%.6f"%best_position[k] for k in range(dim)])
fitness_value = fitness(best_position)
print("fitness of best Student = %.6f" % fitness_value)
 
print("\nEnd TLBO for benchmark 1 function\n")


Begin  teaching learning based optimization on rastrigin function

Goal is to minimize Rastrigin's function in 13 variables
Function has known min = 0.0 at (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
Setting num_particles = 50
Setting max_iter    = 100

Starting TLBO algorithm

Iter = 10 best fitness = -6.567
Iter = 20 best fitness = -15.274
Iter = 30 best fitness = -16.128
Iter = 40 best fitness = -16.195
Iter = 50 best fitness = -16.234
Iter = 60 best fitness = -16.249
Iter = 70 best fitness = -16.249
Iter = 80 best fitness = -16.250
Iter = 90 best fitness = -16.250

TLBO completed


Best Student found:
['-0.499770', '-0.500139', '-0.500177', '-0.500238', '-0.500242', '-0.500142', '-0.499748', '-0.500170', '-0.500656', '-0.500068', '-0.499376', '-0.500399', '-0.499462']
fitness of best Student = -16.249992

End TLBO for benchmark 1 function



In [246]:
#benchmark  2
print("\nBegin  teaching learning based optimization on rastrigin function\n")
dim = 10
fitness = fitness_bench2
 
print("Goal is to minimize Rastrigin's function in " + str(dim) + " variables")
print("Function has known min = 0.0 at (", end="")
for i in range(dim-1):
    print("0, ", end="")
print("0)")
 
num_particles = 50 # population size (no. of student)
max_iter = 100
 
print("Setting num_particles = " + str(num_particles))
print("Setting max_iter    = " + str(max_iter))
print("\nStarting TLBO algorithm\n")
 
 
 
best_position = tlbo(fitness, max_iter, num_particles, dim, 0, 10.0)
 
print("\nTLBO completed\n")
print("\nBest Student found:")
print(["%.6f"%best_position[k] for k in range(dim)])
fitness_value = fitness(best_position)
print("fitness of best Student = %.6f" % fitness_value)
 
print("\nEnd TLBO for benchmark 1 function\n")


Begin  teaching learning based optimization on rastrigin function

Goal is to minimize Rastrigin's function in 10 variables
Function has known min = 0.0 at (0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
Setting num_particles = 50
Setting max_iter    = 100

Starting TLBO algorithm

Iter = 10 best fitness = 0.000
Iter = 20 best fitness = 0.000
Iter = 30 best fitness = 0.000
Iter = 40 best fitness = 0.000
Iter = 50 best fitness = 0.000
Iter = 60 best fitness = 0.000
Iter = 70 best fitness = 0.000
Iter = 80 best fitness = 0.000
Iter = 90 best fitness = 0.000

TLBO completed


Best Student found:
['0.000000', '0.000000', '0.000000', '0.000000', '0.000000', '0.000000', '0.000000', '0.000000', '0.000000', '0.000000']
fitness of best Student = 0.000000

End TLBO for benchmark 1 function



In [245]:
#benchmark  3
print("\nBegin  teaching learning based optimization on rastrigin function\n")
dim = 7
fitness = fitness_bench3
 
print("Goal is to minimize Rastrigin's function in " + str(dim) + " variables")
print("Function has known min = 0.0 at (", end="")
for i in range(dim-1):
    print("0, ", end="")
print("0)")
 
num_particles = 50 # population size (no. of student)
max_iter = 100
 
print("Setting num_particles = " + str(num_particles))
print("Setting max_iter    = " + str(max_iter))
print("\nStarting TLBO algorithm\n")
 
 
 
best_position = tlbo(fitness, max_iter, num_particles, dim, -10.0, 10.0)
 
print("\nTLBO completed\n")
print("\nBest Student found:")
print(["%.6f"%best_position[k] for k in range(dim)])
fitness_value = fitness(best_position)
print("fitness of best Student = %.6f" % fitness_value)
 
print("\nEnd TLBO for benchmark 1 function\n")


Begin  teaching learning based optimization on rastrigin function

Goal is to minimize Rastrigin's function in 7 variables
Function has known min = 0.0 at (0, 0, 0, 0, 0, 0, 0)
Setting num_particles = 50
Setting max_iter    = 100

Starting TLBO algorithm

Iter = 10 best fitness = 3517.422
Iter = 20 best fitness = 3438.825
Iter = 30 best fitness = 3424.032
Iter = 40 best fitness = 3422.115
Iter = 50 best fitness = 3421.968
Iter = 60 best fitness = 3421.965
Iter = 70 best fitness = 3421.965
Iter = 80 best fitness = 3421.965
Iter = 90 best fitness = 3421.965

TLBO completed


Best Student found:
['1.151928', '1.151928', '1.151927', '1.151927', '1.151928', '1.151925', '1.151926']
fitness of best Student = 3421.964875

End TLBO for benchmark 1 function



In [275]:
#benchmark  3
print("\nBegin  teaching learning based optimization on rastrigin function\n")
dim = 8
fitness = fitness_bench4
 
print("Goal is to minimize Rastrigin's function in " + str(dim) + " variables")
print("Function has known min = 0.0 at (", end="")
for i in range(dim-1):
    print("0, ", end="")
print("0)")
 
num_particles = 50 # population size (no. of student)
max_iter = 100
 
print("Setting num_particles = " + str(num_particles))
print("Setting max_iter    = " + str(max_iter))
print("\nStarting TLBO algorithm\n")
 
 
 
best_position = tlbo(fitness, max_iter, num_particles, dim, -10.0, 10.0)
 
print("\nTLBO completed\n")
print("\nBest Student found:")
print(["%.6f"%best_position[k] for k in range(dim)])
fitness_value = fitness(best_position)
print("fitness of best Student = %.6f" % fitness_value)
 
print("\nEnd TLBO for benchmark 1 function\n")


Begin  teaching learning based optimization on rastrigin function

Goal is to minimize Rastrigin's function in 8 variables
Function has known min = 0.0 at (0, 0, 0, 0, 0, 0, 0, 0)
Setting num_particles = 50
Setting max_iter    = 100

Starting TLBO algorithm

Iter = 10 best fitness = -6367.493
Iter = 20 best fitness = -7280.000
Iter = 30 best fitness = -7280.000
Iter = 40 best fitness = -7280.000
Iter = 50 best fitness = -7280.000
Iter = 60 best fitness = -7280.000
Iter = 70 best fitness = -7280.000
Iter = 80 best fitness = -7280.000
Iter = 90 best fitness = -7280.000

TLBO completed


Best Student found:
['-10.000000', '-10.000000', '-10.000000', '-10.000000', '-10.000000', '-10.000000', '-10.000000', '-10.000000']
fitness of best Student = -7280.000000

End TLBO for benchmark 1 function



In [131]:
#pip install benchmark_functions

In [142]:
import benchmark_functions as bf 

def fitness_bench2(position):
    fitness_value = 0.0
    for i in range(len(position)):
        xi = position[i]
        fitness_value += bf.Rastrigin(xi, n_dimensions=3)
    return fitness_value

In [134]:
import benchmark_functions as bf

bf.Rastrigin()

In [186]:
import numpy as np
from sklearn.utils import check_random_state

class TeachingLearningBasedOptimizer(object):

    def __init__(self, f, lower_bound, upper_bound, n_population, random_state=None):

        self.f = f
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.pidx = list(range(n_population))
        self.m = lower_bound.shape[0]  
        self.random_state = check_random_state(random_state)

        self.population_ = self.random_state.rand(n_population, self.m) * (upper_bound - lower_bound) + lower_bound
        self.fitness_ = np.apply_along_axis(self.f, 1, self.population_)

        self.bestidx_ = np.argmin(self.fitness_)
        self.bestcosts_ = [ self.fitness_[self.bestidx_] ]

    def best(self, num=1):
     
        bestidxs = np.argsort(self.fitness_)[:num]
        return self.population_[bestidxs], self.fitness_[bestidxs]

    def next(self):

        mean = np.nanmean(self.population_, axis=0) 
        rs = self.random_state

        teacher = np.argmin(self.fitness_)

        for i in self.pidx:
            T_F = rs.randint(1, 3)
            r_i = rs.rand(self.m)

            new_solution = self.population_[i] + (r_i * (teacher - (T_F * mean)))
            new_solution = np.minimum(np.maximum(new_solution, self.lower_bound), self.upper_bound)

            new_fitness = self.f(new_solution)

            if new_fitness < self.fitness_[i]:
                self.population_[i] = new_solution
                self.fitness_[i] = new_fitness

        for i in self.pidx:

            j = rs.choice(self.pidx[:i] + self.pidx[(i + 1):], 1)
            r_i = rs.rand(self.m)

            if self.fitness_[i] < self.fitness_[j]:
                new_solution = self.population_[i] + r_i * (self.population_[i] - self.population_[j]).flatten()
            else:
                new_solution = self.population_[i] + r_i * (self.population_[j] - self.population_[i]).flatten()

            new_solution = np.minimum(np.maximum(new_solution, self.lower_bound), self.upper_bound)
            new_fitness = self.f(new_solution)
            
            if new_fitness < self.fitness_[i]:
                self.population_[i] = new_solution
                self.fitness_[i] = new_fitness

        self.bestidx_ = np.argmin(self.fitness_)
        self.bestcosts_.append(self.fitness_[self.bestidx_])

TLBO = TeachingLearningBasedOptimizer 

In [187]:
def f(position):
    fitness_value = 0.0
    for i in range(len(position)):
        xi = position[i]
        fitness_value += (xi * xi) - (10 * math.cos(2 * math.pi * xi)) + 10
    return fitness_value

In [188]:
TLBO(f, -10.0, 10.0, 50)

AttributeError: 'float' object has no attribute 'shape'

In [284]:
#pip install pyOpt

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np 
from random import randint 

from pyOpt import Optimization



def p_vessel(x):
    Ts = x[0]
    Th = x[1]
    R = x[2]
    L = x[3]
    f = 0.6224.Ts*R*L + 1.2281*Th*R**2 + 3.1661*(Ts**2)*L + 19.84*(Ts**2)*R
    g = [0.0]*4
    g[0] = -Ts + 0.0193*R
    g[1] = -Th + 0.00954*R
    g[2] = -np.pi*(R**2)*L -(4/3)*np.pi*R**3 +1296000
    g[3] = L - 240
    return f, g, fail

t = randint(0, 99)
Rr = randint(10, 200)
Ll = randint(10, 200)
d = 0.0625

opt = Optimization("Pressure Vessel Design", p_vessel)
opt.addVar('Ts', 'c', lower = d, upper = 99*d, value = d*t)
opt.addVar('Th', 'c', lower = d, upper = 99*d, value = d*t)
opt.addVar('R', 'c', lower = 10.0, upper = 200.0, value = Rr)

ModuleNotFoundError: No module named 'pyOpt'